In [1]:
%%time

import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
from bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.GPBO_Class_fxns import * #Fix this later

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: user 2.36 s, sys: 306 ms, total: 2.67 s
Wall time: 2.7 s


## Define Method and DateTime

In [2]:
#Generate Method (except last part)
meth_name = Method_name_enum(2)
method = GPBO_Methods(meth_name)

print("Method Name: ", method.method_name.name)
print("Emulator?: ", method.emulator)
print("Obj Func: ", method.obj.name)
print("Sparse Grid?: ", method.sparse_grid)

#Set Date and Time
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S)")
# print("Date and Time: ", timestampStr)
# DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M-%S%p")
DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M")
DateTime = None ##For Testing

Method Name:  B1
Emulator?:  False
Obj Func:  LN_OBJ
Sparse Grid?:  False


## Initialize Simulator

In [3]:
def simulator_helper_test_fxns(cs_name, indecies_to_consider, noise_mean, noise_std, case_study_parameters):
    """
    Sets the model for calculating y based off of the case study identifier.

    Parameters
    ----------
    cs_name: Class, The name/enumerator associated with the case study being evaluated

    Returns
    -------
    calc_y_fxn: function, the function used for calculation is case study cs_name.name
    """
    #Note: Add your function name from GPBO_Class_fxns.py here
    if cs_name.value == 1:      
        theta_ref = np.array([1.0, -1.0])
        theta_names = ['theta_1', 'theta_2']
        bounds_x_l = [-2]
        bounds_x_u = [2]
        bounds_theta_l = [-2, -2]
        bounds_theta_u = [ 2,  2]
        calc_y_fxn = calc_cs1_polynomial
        
    elif cs_name.value == 2:     
        theta_ref = np.array([-200,-100,-170,15,-1,-1,-6.5,0.7,0,0,11,0.6,-10,-10,-6.5,0.7,1,0,-0.5,-1,0,0.5,1.5,1])
                             
        theta_names = ['A_1', 'A_2', 'A_3', 'A_4', 'a_1', 'a_2', 'a_3', 'a_4', 'b_1', 'b_2', 'b_3', 'b_4', 'c_1', 
                       'c_2', 'c_3', 'c_4', 'x0_1', 'x0_2', 'x0_3', 'x0_4', 'x1_1', 'x1_2', 'x1_3', 'x1_4']
        bounds_x_l = [-1.5, -0.5]
        bounds_x_u = [1, 2]
        bounds_theta_l = [-300,-200,-250, 5,-2,-2,-10, -2, -2,-2,5,-2,-20,-20, -10,-1 ,-2,-2,-2, -2,-2,-2,0,-2]
        bounds_theta_u = [-100,  0, -150, 20,2, 2, 0,  2,  2,  2, 15,2, 0,0   , 0,  2, 2,  2, 2, 2 ,2 , 2, 2,2]
        calc_y_fxn = calc_muller
        
    else:
        raise ValueError("self.CaseStudyParameters.cs_name.value must exist!")

    return Simulator(indecies_to_consider, 
                     theta_ref,
                     theta_names,
                     bounds_theta_l, 
                     bounds_x_l, 
                     bounds_theta_u, 
                     bounds_x_u, 
                     noise_mean,
                     noise_std,
                     case_study_parameters,
                     calc_y_fxn)

## Initialize Case Study and Simulator

In [4]:
cs_name  = CS_name_enum(2)
print("CS Name: ", cs_name.name)

# indecies_to_consider = list(range(0, 2)) #This is what changes for different subproblems of CS1
indecies_to_consider = list(range(4, 12)) #This is what changes for different subproblems of CS1

ep0 = 1
sep_fact = 0.8
normalize = False
lhs_gen_theta = True
eval_all_pairs = False
noise_mean = 0
noise_std = 0.01
noise_std = 0
kernel = Kernel_enum(1)
set_lenscl = 1
outputscl = False
retrain_GP = 2
GP_train_iter = 300
bo_iter_tot = 3
bo_run_tot = 2
save_fig = False
save_data = False
num_data = None
seed = 3

cs_params = CaseStudyParameters(cs_name, ep0, sep_fact, normalize, eval_all_pairs, bo_iter_tot, bo_run_tot, 
                         save_fig, save_data, DateTime, seed)

simulator = simulator_helper_test_fxns(cs_name, indecies_to_consider, noise_mean, noise_std, cs_params)

CS Name:  CS2


## Test Data

In [5]:
#Generate exp_data
num_x_data = 5
gen_meth_x = Gen_meth_enum(2)
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x)

In [6]:
#Test dimension checkers
# assert exp_data.get_num_theta() == 5
# assert exp_data.get_dim_theta() == 2
# assert exp_data.get_num_x_vals() == 5
# assert exp_data.get_dim_x_vals() == 1

In [7]:
#Test normalize
# scaled_exp_data = exp_data.norm_feature_data()
# assert np.allclose(np.array([[0.75, 0.25], 
#                              [0.75, 0.25], 
#                              [0.75, 0.25], 
#                              [0.75, 0.25], 
#                              [0.75, 0.25]]), scaled_exp_data.theta_vals) == True
# assert np.allclose(np.array([[0.0], 
#                              [0.25], 
#                              [0.5], 
#                              [0.75], 
#                              [1]]), scaled_exp_data.x_vals) == True

In [8]:
#Test unnormalize
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x)
scaled_exp_data = exp_data.norm_feature_data()
exp_data_reg = scaled_exp_data.unnorm_feature_data()
assert np.allclose(exp_data.theta_vals, exp_data_reg.theta_vals)
assert np.allclose(exp_data.x_vals, exp_data_reg.x_vals)

In [9]:
# test train_test_idx_split
train_idx, test_idx = exp_data_reg.train_test_idx_split(cs_params)
union_set = set(train_idx).union(test_idx)
assert (len(train_idx) + len(test_idx) == exp_data_reg.get_num_theta())
assert set(range(exp_data_reg.get_num_theta())).issubset(union_set)